In [ ]:
val df = spark.read.json("gs://cmuccpublicdatasets/twitter/dataset/").cache

In [1]:
val df1 = spark.read.json("gs://cmuccpublicdatasets/twitter/dataset/part-r-00000.gz").cache


In [2]:
import java.nio.file.{Files, Paths}
import scala.jdk.CollectionConverters._

val dirPath = "gs://cmuccpublicdatasets/twitter/dataset/"

val fileList = Files.list(Paths.get(dirPath)).iterator().asScala
for (path <- fileList) {
  println(path.getFileName)
}

In [3]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._

// Assuming you've already initialized SparkSession as 'spark'
val basePath = "gs://cmuccpublicdatasets/twitter/dataset/"

// Generate file paths
val fileNames = (0 until 50).map { i =>
  f"$basePath%spart-r-${i}%05d.gz"
}

// Read the first file to initialize df2
var df2 = spark.read.json(fileNames.head).cache()

// Read the remaining files and union them with df2
fileNames.tail.foreach { filePath =>
  val nextDf = spark.read.json(filePath)
  df2 = df2.unionByName(nextDf, allowMissingColumns = true)
}
df2 = df2.cache
// Now, df2 contains the data from the first 50 files

In [4]:
df2.show(1)

In [5]:
basePath = "gs://cmuccpublicdatasets/twitter/dataset/"
fileNames = [f"{basePath}part-r-{str(i).zfill(5)}.gz" for i in range(50)]
print(fileNames)

In [6]:
%python
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("Read Multiple JSON Files") \
    .getOrCreate()
    
# Assuming SparkSession has already been initialized as 'spark'
basePath = "gs://cmuccpublicdatasets/twitter/dataset/"

basePath = "gs://cmuccpublicdatasets/twitter/dataset/"
fileNames = [f"{basePath}part-r-{str(i).zfill(5)}.gz" for i in range(50)]
print(fileNames)
# Read all files at once
df2 = spark.read.json(fileNames)


In [7]:
%python
df2.show(1)

In [8]:
%python
from pyspark.sql.functions import col, concat_ws, size

valid_tweets = df2.filter(
    (col("id").isNotNull() | col("id_str").isNotNull()) &
    (col("user.id").isNotNull() | col("user.id_str").isNotNull()) &
    col("created_at").isNotNull() &
    (col("text").isNotNull() & (col("text") != "")) &
    (col("entities.hashtags").isNotNull() & (size(col("entities.hashtags")) > 0)) &
    col("lang").isin("ar", "en", "fr", "in", "pt", "es", "tr", "ja")
).dropDuplicates(["id"])

new_df = valid_tweets.withColumn("hashtags", concat_ws(", ", col("entities.hashtags.text"))).select(
    col("created_at"),
    col("hashtags"),
    col("id"),
    col("in_reply_to_user_id").alias("reply_uid"),
    col("retweeted_status.id").alias("retweet_tid"),
    col("retweeted_status.user.id").alias("retweet_uid"),
    col("text"),
    col("user.id").alias("uid"),
    col("user.screen_name").alias("user_screen_name"),
    col("user.description").alias("user_description")
)


In [9]:
%python
import pandas as pd

# Set credentials
jdbc_port = "3306"
jdbc_database = "etl_db"
jdbc_username = "etl_user"
jdbc_password = "PAYTON401gzt"  # The password you set
jdbc_hostname = "34.139.20.62"  # The external IP address of your MySQL DB

jdbc_url = f"jdbc:mysql://{jdbc_hostname}:{jdbc_port}/{jdbc_database}?user={jdbc_username}&password={jdbc_password}"


# Set connection properties
properties = {
    "user": jdbc_username,
    "password": jdbc_password,
    "driver": "com.mysql.jdbc.Driver",
    "useServerPrepStmts": "false",  # Note this configuration
    "rewriteBatchedStatements": "true",  # Note this configuration
}

new_df.write.jdbc(url=jdbc_url, table="tweets", mode="overwrite", properties=properties)

In [10]:
df2.show(10)

In [11]:
df1.show(5)

In [12]:
val sample_df = df.limit(100000)

In [13]:
val validTweets = df3.filter(
  (col("id").isNotNull || col("id_str").isNotNull) &&
  (col("user.id").isNotNull || col("user.id_str").isNotNull) &&
  col("created_at").isNotNull &&
  (col("text").isNotNull && col("text") =!= "") &&
  (col("entities.hashtags").isNotNull && size(col("entities.hashtags")) > 0) &&
  col("lang").isin("ar", "en", "fr", "in", "pt", "es", "tr", "ja")
)
.dropDuplicates("id")


In [14]:
val newDf = validTweets
.withColumn("hashtags", concat_ws(", ", col("entities.hashtags.text")))
.select(
    col("created_at"),
    col("hashtags"),
    col("id"),
    col("in_reply_to_user_id").as("reply_uid"),
    col("retweeted_status.id").as("retweet_tid"),
    col("retweeted_status.user.id").as("retweet_uid"),
    col("text"),
    col("user.id").as("uid"),
    col("user.screen_name").as("user_screen_name"),
    col("user.description").as("user_description")
)

In [15]:
import java.util.Properties
import java.sql.DriverManager

// set credentials
val jdbcPort = "3306"
val jdbcDatabase = "etl_db"
val jdbcUsername = "etl_user"
val jdbcPassword = "PAYTON401gzt"   // the password you set
val jdbcHostname = "34.139.20.62"   // the external IP address of you MySQL DB

val jdbcUrl =s"jdbc:mysql://$jdbcHostname:$jdbcPort/$jdbcDatabase"

val driverClass = "com.mysql.jdbc.Driver"
Class.forName(driverClass)  // check jdbc driver

// set connection properties
val connectionProperties = new Properties()
connectionProperties.put("user", s"$jdbcUsername")
connectionProperties.put("password", s"$jdbcPassword")
connectionProperties.setProperty("Driver", driverClass)
connectionProperties.setProperty("useServerPrepStmts", "false")    // take note of this configuration, and understand what it does
connectionProperties.setProperty("rewriteBatchedStatements", "true")  // take note of this configuration, and understand what it does

// first drop the table if it already exists
val connection = DriverManager.getConnection(jdbcUrl, jdbcUsername, jdbcPassword)
assert(!connection.isClosed)
val stmt = connection.createStatement()
stmt.executeUpdate("drop table if exists tweets")



In [16]:
// write the dataframe to a table called "biz_review"
newDf.write.jdbc(jdbcUrl, "tweets", connectionProperties)

In [17]:
%sh
mysql -h34.139.20.62 -pPAYTON401gzt -uetl_user -e 'use etl_db; select * from tweets limit 1;'

Official Write Trial

In [19]:
val validTweets = sample_df.filter(
  (col("id").isNotNull || col("id_str").isNotNull) &&
  (col("user.id").isNotNull || col("user.id_str").isNotNull) &&
  col("created_at").isNotNull &&
  (col("text").isNotNull && col("text") =!= "") &&
  (col("entities.hashtags").isNotNull && size(col("entities.hashtags")) > 0) &&
  col("lang").isin("ar", "en", "fr", "in", "pt", "es", "tr", "ja")
)
.dropDuplicates("id_str")


In [20]:
val newDf_all = validTweets
.withColumn("hashtags", concat_ws(", ", col("entities.hashtags.text")))
.select(
    col("created_at"),
    col("hashtags"),
    col("id"),
    col("in_reply_to_user_id").as("reply_uid"),
    col("retweeted_status.id").as("retweet_tid"),
    col("retweeted_status.user.id").as("retweet_uid"),
    col("text"),
    col("user.id").as("uid"),
    col("user.screen_name").as("user_screen_name"),
    col("user.description").as("user_description")
)

In [21]:
import java.util.Properties
import java.sql.DriverManager

// set credentials
val jdbcPort = "3306"
val jdbcDatabase = "etl_db"
val jdbcUsername = "etl_user"
val jdbcPassword = "PAYTON401gzt"   // the password you set
val jdbcHostname = "34.139.20.62"   // the external IP address of you MySQL DB

val jdbcUrl =s"jdbc:mysql://$jdbcHostname:$jdbcPort/$jdbcDatabase"

val driverClass = "com.mysql.jdbc.Driver"
Class.forName(driverClass)  // check jdbc driver

// set connection properties
val connectionProperties = new Properties()
connectionProperties.put("user", s"$jdbcUsername")
connectionProperties.put("password", s"$jdbcPassword")
connectionProperties.setProperty("Driver", driverClass)
connectionProperties.setProperty("useServerPrepStmts", "false")    // take note of this configuration, and understand what it does
connectionProperties.setProperty("rewriteBatchedStatements", "true")  // take note of this configuration, and understand what it does

// first drop the table if it already exists
val connection = DriverManager.getConnection(jdbcUrl, jdbcUsername, jdbcPassword)
assert(!connection.isClosed)
val stmt = connection.createStatement()
stmt.executeUpdate("drop table if exists tweets")

// write the dataframe to a table called "biz_review"
newDf_all.write.jdbc(jdbcUrl, "tweets", connectionProperties)

### Test counts

In [ ]:
val df1 = spark.read.json("gs://cmuccpublicdatasets/twitter/dataset/part-r-00000.gz").cache

In [ ]:
val correct_df1 = spark.read.json("microservice3_ref.txt").cache 

In [ ]:
correct_df1.count()

import org.apache.spark.sql.functions._

val filtered_tweets_df = df1.filter(
  (col("id").isNotNull || col("id_str").isNotNull) &&
  (col("user.id").isNotNull || col("user.id_str").isNotNull) &&
  col("created_at").isNotNull &&
  (col("text").isNotNull && col("text") =!= "") &&
  (col("entities.hashtags").isNotNull && size(col("entities.hashtags")) > 0) &&
  col("lang").isin("ar", "en", "fr", "in", "pt", "es", "tr", "ja")
)


In [ ]:
filtered_tweets_df.count()

### Histogram

import org.apache.spark.sql.functions._

val hashtagCounts = df
  .withColumn("hashtag", explode($"hashtags")) 
  .groupBy("hashtag")
  .count() 
  .withColumn("logCount", log10($"count")) 
  .orderBy($"count".desc) 

In [ ]:
val hashtagFrequencies = hashtagCounts.write.csv("hashtag_frequencies.csv")


In [ ]:
val xValues = hashtagFrequencies.map(_.getString(0)) 
val yValues = hashtagFrequencies.map(_.getDouble(2))


In [ ]:
%python
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
%python
hashtag = pd.read_csv('hashtag_frequencies.csv')
df_sorted = hashtag.sort_values(by='count', ascending=False)
# Plotting
plt.figure(figsize=(10, 6))

# We use log=True to indicate we want the y-axis to be on a log scale.
# If you want to plot the logCount directly, ensure it's correctly calculated.
plt.hist(df_sorted['logCount'], bins=50, log=True)

plt.title('Histogram of Hashtag Frequencies on a Log Scale')
plt.xlabel('Log of Hashtag Frequency')
plt.ylabel('Number of Hashtags')
plt.grid(True, which="both", ls="--", linewidth=0.5)

plt.show()
